mplementatie van Quantum Phase Estimation (QPE) werkt voor het waterstofmolecuul (H₂), met volledige verwijzing naar de relevante implementaties in jouw repositories `BramDo/qiskit` en `BramDo/qiskit-nature`. Ik zal de fysische principes, de keuze voor poorten en representaties, de betrouwbaarheid van resultaten, en eventuele beperkingen voor grotere systemen behandelen.




# Quantum Phase Estimation voor het H₂-molecuul: Toelichting van implementatie

## Fysische en kwantummechanische principes van H₂-simulatie

De simulatie van het waterstofmolecuul (H₂) met Quantum Phase Estimation (QPE) is gebaseerd op de kwantummechanische beschrijving van moleculaire binding. In de Born-Oppenheimer-benadering beschouwen we de kernen als vast (op een bepaalde afstand) terwijl we de elektronen kwantummeganisch simuleren. De Hamiltoniaan van H₂ omvat de kinetische energie van de elektronen, de coulomb-interacties tussen elektronen onderling en met de atoomkernen, plus de coulomb-afstoting tussen de kernen (de *nucleaire repulsie* als constante term). Deze Hamiltoniaan is Hermitisch en zijn laagste eigenwaarde correspondeert met de grondtoestandenergie van H₂. QPE maakt gebruik van het principe dat een tijdsevolutie volgens de Schrödingervergelijking faseinformatie draagt over deze eigenwaarde. Als we een kwantisysteem in een (benaderde) grondtoestand $|\psi\_0\rangle$ voorbereiden, zal toepassen van de unitaire operator $U(t) = e^{-iHt}$ een fase $e^{-iE\_0 t}$ in de toestandsvector introduceren. Hierin is $E\_0$ de (onbekende) energie van de grondtoestand. QPE exploiteert dit door interferentie en Fourier-analyse: door gecontroleerde tijdsevolutie en een inverse Quantum Fourier Transformatie (QFT) kan de fase – en daarmee de energie – worden uitgelezen als meetbare qubit-bits. Cruciaal is dat de beginstate een overlap heeft met de eigenstate waarvan we de energie willen weten; in deze implementatie wordt de Hartree-Fock benadering van de grondtoestand als initieel kwantumtoestand gebruikt. Deze Hartree-Fock-toestand $|\Phi\_{\text{HF}}\rangle$ is een enkelvoudige determinantenbenadering van de grondtoestand, die voor H₂ zeer dicht bij de echte grondtoestand ligt. Daardoor heeft $|\Phi\_{\text{HF}}\rangle$ een grote overlap met de echte grondtoestand, wat betekent dat een QPE-metingen met hoge waarschijnlijkheid de fase van de grondtoestandenergie opleveren.

## Qubit-Hamiltoniaan van H₂: opstelling en mapping

Om QPE uit te voeren, moeten we de elektronische Hamiltoniaan van H₂ naar een qubitoperator vertalen. Deze implementatie gebruikt Qiskit Nature’s hulpmiddelen om de Hamiltoniaan op te stellen. Eerst genereert de `PySCFDriver` de elektronische integralen voor H₂ bij een gegeven internucleaire afstand. Deze driver levert een `ElectronicStructureProblem` (`problem`), waarvan de methode `second_q_ops()` een lijst van tweede-gequantiseerde operatoren teruggeeft. De eerste hiervan is de elektronische Hamiltoniaan van het systeem. In formulevorm is dit bijvoorbeeld:

$H_{\text{electr}} = \sum_{p,q} h_{pq} a_p^\dagger a_q + \frac{1}{2}\sum_{p,q,r,s} h_{pqrs} a_p^\dagger a_q^\dagger a_r a_s,$

waar $a\_p^\dagger, a\_q$ fermionische creatie- en annihilatieoperatoren voorstellen voor de orbitals. De nucleaire repulsie-energie wordt meestal apart bijgehouden (`problem.nuclear_repulsion_energy`), omdat deze een constante (identiteits)term in de Hamiltoniaan is.

Vervolgens moet deze fermionische Hamiltoniaan gemapt worden naar qubits. In de gegeven implementatie wordt daarvoor de **Parity-mapper** gebruikt (`ParityMapper`). De Parity-mapping is een fermion-to-qubit-transformatie die rekening houdt met het pariteitspatroon van bezette orbitalen. Een belangrijk voordeel is dat ParityMapping, mits voorzien van het totaal aantal deeltjes, automatisch een reductie van twee qubits kan uitvoeren (de zogeheten two-qubit reduction). Concreet: omdat bij H₂ de spinorbitalen in paren (spin-up/spin-down) geordend zijn, weet de `ParityMapper` op basis van het aantal elektronen welke $Z\_2$-symmetrieën (pariteit van het aantal elektronen per spin-sector) gefixeerd zijn. Hij tapeert dan twee qubits weg (meestal de middelste en laatste qubit) en selecteert zo automatisch de correcte symmetriesector van de operator. In code gebeurt dit door de mapper te initialiseren met `num_particles` (het aantal $\alpha$- en $\beta$-elektronen) en vervolgens de tweede-gequantiseerde Hamiltoniaan te mappen naar een `SparsePauliOp` (een som van Pauli-termen) via `mapper.map(problem.second_q_ops()[0])`. Voor H₂ (2 elektronen in 4 spinorbitalen) resulteert dit na two-qubit reduction in een qubit-Hamiltoniaan op slechts **2 qubits**. De Hamiltoniaan wordt daarmee voorgesteld als een som van Pauli-operatoren op 2 qubits, bijvoorbeeld een combinatie van termen als $ZI$, $IZ$, $ZZ$, $XX$ etc. met bijbehorende coëfficiënten.

*(Opmerking: Een eventuele **FreezeCoreTransformer** is geïmporteerd in de code maar voor H₂ feitelijk niet nodig. Freeze-core is een techniek om inactieve (“kern-”)orbitalen te verwijderen uit het probleem om qubits te besparen. Bij H₂ in een minimale basis zijn er geen diepgelegen kernorbitalen om te bevriezen – beide elektronen bevinden zich in de valentie-orbitalen, dus de transform niet gebruikt in deze simulatie. In grotere moleculen zou FreezeCore de Hamiltoniaan eerst aanpassen door kernorbitalen (die altijd dubbel bezet zijn) te verwijderen, alvorens de mapper toe te passen.)*

## Gecontroleerde unitaire evolutie met **PauliEvolutionGate**

Het kernidee van QPE is om de unitary $U = e^{-iH t}$ herhaaldelijk gecontroleerd toe te passen, waarbij $H$ de qubit-Hamiltoniaan is. In deze implementatie wordt $t$ in opeenvolgende machten van 2 gekozen (1, 2, 4, 8, ...). De realisatie hiervan gebeurt efficiënt via Qiskit’s `PauliEvolutionGate`. Deze klasse genereert een poort die een gegeven Pauli-operator (of som van Pauli’s) exponentieert voor een bepaalde evolutietijd. In wiskundige notatie implementeert `PauliEvolutionGate(operator, time=t)` de operator $U(t) = \exp(-i\,t\,H)$, waarbij `operator` bijvoorbeeld de `SparsePauliOp`-voorstelling van de H₂-Hamiltoniaan is.

In de code wordt eerst een quantumregister met $n$ systeem-qubits (hier $n=2$) en $m$ ancilla-qubits aangemaakt, met $m=12$ in dit geval. De eerste ancilla-qubit wordt gebruikt om een **gecontroleerde** evolutie toe te passen: voor elke ancilla $i$ (met $i=0,...,m-1$) wordt de unitaire evolutie voor tijd $2^i$ geconditioneerd op de toestand van ancilla $i$. Dit gebeurt door de evolutiepoort te halen via `PauliEvolutionGate(qubit_op, time=2**i)` en daar `.control()` van te maken, waarna deze gecontroleerde poort aan het circuit wordt toegevoegd. Qiskit plaatst de control op de opgegeven ancilla-qubit en laat de evolutie op de systeem-qubits werken. Het resultaat is dat ancilla $i$ de fase-informatie opvangt die overeenkomt met $2^i$ keer de evolutietijd. Voor H₂ met 12 ancillas betekent dit dat we effectief de faseterm $E\_0 \cdot 2^i$ (in eenheid van rad/s, bij $t=1$) accumuleren op de respectievelijke qubitfase. Alle ancilla-qubits worden vóór deze evoluties in een superpositie gezet door Hadamard-poorten (`qc.h(range(n_ancillas))`), zodat elke control-lijn in staat is de evolutie zowel wel als niet uit te voeren en zo een interferentiepatroon opbouwt over alle $2^m$ mogelijke fase-basisstates.

Qiskit’s `PauliEvolutionGate` zorgt intern voor de decompositie van $e^{-iHt}$ in daadwerkelijke quantumpoorten. Standaard gebeurt dit met een eerste-orde Trotter-productformule (Lie-Trotter) als synthesemethode. Dat wil zeggen: als $H$ uit meerdere niet-commuterende Pauli-termen bestaat $H = \sum\_j P\_j$, benadert de gate $e^{-iH t}$ als $e^{-iP\_1 t}e^{-iP\_2 t}\cdots e^{-iP\_N t}$ (één Trotter-stap). Voor een klein molecuul als H₂, waar $H$ relatief weinig termen heeft en de evolutietijdstappen in QPE (bijvoorbeeld $t = 2^{11}$ als grootste) nog beheersbaar zijn, levert deze benadering acceptabele resultaten. Er ontstaat wel een kleine Trotter-fout die groeit met $t$ en met het niet-commuterende karakter van de termen. Dit is zichtbaar in de resultaten als een lichte afwijking van de echte energie (meer hierover later). Desgewenst had men de precisie kunnen verhogen door de synthesemethode te wijzigen of meer Trotter-stappen te gebruiken, ten koste van een dieper circuit. In deze demo-volgt men echter de standaardinstelling.

Kortom, met de combinatie van Hadamard-gates op de ancilla’s en de reeks gecontroleerde $U(2^i)$ evoluties wordt het standaard QPE-circuit opgebouwd. Na deze evolutiefase bevatten de ancilla-qubits de fase-informatie (in binair vorm) van de energie van de toestand.

## Fase-uitvoer en grondtoestandenergie uit QPE

Na de gecontroleerde evoluties voert men de inverse Quantum Fourier Transformatie uit op de ancilla-registers (`qc.append(QFT(n_ancillas, inverse=True), range(n_ancillas))` in de code). De inverse QFT vertaalt de in de amplitudes gesuperponeerde fase-informatie naar een toestand waarin elke ancilla-qubit een klassiek bit van het fasegetal bevat. Met andere woorden: als de toestand van de systeem-qubits een eigenstaat $|\psi\_E\rangle$ van $H$ was, zal na inverse QFT de toestand van de $m$ ancilla’s zeer dicht bij de binaire representatie van $\phi = E \cdot t / (2\pi)$ liggen, waarbij $E$ de eigenenergie is (aangenomen dat $U(t) = e^{-iHt}$ zoals hier) en $t=1$ in ons geval. De implementatie meet vervolgens alle ancilla-qubits in de computational-basis (`qc.measure(range(n_ancillas), range(n_ancillas))`) en verkrijgt zo een bitstring van lengte 12 als uitkomst per shot.

Omdat de grondtoestandinitiële $|\Phi\_{\text{HF}}\rangle$ vrijwel helemaal in de grondtoestandcomponent zit, zal het gemeten bitstring met hoogste waarschijnlijkheid overeenkomen met de fase van de grondtoestandenergie $E\_0$. Ter interpretatie worden de meetresultaten als volgt verwerkt: elke bitstring $b$ wordt omgezet naar een decimaal getal $\text{int}(b)$ en gedeeld door $2^m$ om de gerapporteerde fasefractie $\phi$ te krijgen. Deze fase relateert aan de energie via $\phi \approx -E \cdot t / (2\pi)$ (het minteken is er omdat QPE hier $U = e^{-iH}$ gebruikte, zie hierboven). Met $t=1$ geldt eenvoudig $E \approx - 2\pi \phi$. In de code zien we dat voor elk bitstring de energie wordt teruggerekend als `energy = -1 * (phase * (2 * np.pi) / evolution_time)`. Hier is `evolution_time` = 1, dus feitelijk $E = -2\pi \phi$. Bijvoorbeeld, als de fasebits `000000000000` gemeten zouden worden (binair 0) correspondeert dat met $\phi=0$ en energie 0 – voor H₂ onrealistisch. Een meetuitkomst zoals `101010101010` (binair 2730) zou $\phi\approx2730/4096=0.6665$ geven en daarmee $E\approx -2\pi(0.6665)\approx -4.188$, enzovoort. In de praktijk zullen de uitkomsten clusteren rond de fase die overeenkomt met de grondtoestandenergie.

De implementatie verzamelt de frequenties (`counts`) van elk gemeten bitstring over vele herhalingen en berekent voor de zekerheid een gewogen verwachtingswaarde van de energie: $\langle E \rangle = \sum\_b P(b) E(b)$, met $P(b)$ de relatieve frequentie van bitstring $b$. Deze **verwachtingswaarde** zou in ideale omstandigheden gelijk moeten zijn aan $E\_0$ als $|\Phi\_{\text{HF}}\rangle$ exact de grondtoestand is (dan is er maar één bitstring-uitkomst). In gevallen waar de initiële toestand nog een fractie van een aangeslagen toestand bevat, zou de gemeten verdeling twee pieken kunnen vertonen (één voor grondtoestand, één voor een hogere energie) en dan ligt de verwachting ergens daartussen. In deze H₂-simulatie is de grondtoestandskans doorgaans dominant, waardoor de berekende $\langle E \rangle$ dicht bij de grondtoestandenergie ligt. Men vergelijkt deze QPE-energie vervolgens met de exacte waarde ter validatie.

Concreet wordt bij elke molecuulafstand het volgende gerapporteerd: de exacte grondtoestandenergie (uit klassiekere berekening) versus de via QPE verkregen energie, en het verschil daartussen. Voor de exacte waarde is de nucleaire repulsie uit de totale energie gehaald om eerlijk te vergelijken met de QPE-uitkomst (die alleen de elektronische bijdrage omvat). We zien dat QPE de energiewaarde correct in de orde van grootte heeft en vaak dicht benadert: bijvoorbeeld bij een H–H afstand van 0,8 Å is de exacte grondtoestandenergie (electronsiche deel) $-1.7956$ Hartree en QPE levert $-1.7960$ Hartree – een fout van slechts 0,02%. In sommige gevallen treedt een iets grotere afwijking op, oplopend tot circa 5–7% fout bij de slechtste punten (bijv. 1,6 Å). Deze afwijkingen komen voort uit de besproken factoren: beperkte overlap van de HF-toestand bij dissociatieve geometrie (waardoor kans op meten van een hogere energie bestaat) én de benaderingen in de gecontroleerde evolutie (Trotterfout die zich sterker manifesteert bij langere evolutietijden of grotere energiestappen).

## Toepassing op H₂ versus grotere moleculen

Het is geen toeval dat H₂ een veelgebruikt voorbeeld is voor quantum-simulatie: dit diatomische molecuul is het eenvoudigste niet-triviale molecuulsysteem en vergt het allerminste aan quantumresources. Na mapping en symmetriereductie vereist H₂ in minimale basis slechts 2 qubits voor de werkelijke kwantumsimulatie van de elektronen, wat uitzonderlijk laag is. Bovendien is de Hartree-Fock benadering voor H₂ (zeker rond de evenwichtsafstand \~0,74 Å) bijna exact – er is nauwelijks elektronische correlatie aanwezig die HF niet opvangt. Dit betekent dat de initiële state $|\Phi\_{\text{HF}}\rangle$ praktisch al de grondtoestand is, zodat QPE meteen met hoge nauwkeurigheid het juiste eigenenergie-fasebitstring meet. De benodigde circuitdiepte blijft beperkt: de H₂-Hamiltoniaan heeft relatief weinig Pauli-termen en doordat we slechts één Trotterstap gebruiken, bestaat de ongecontroleerde evolutie $e^{-iHt}$ uit een handvol exponentiële rotaties voor deze termen. Het controleerbaar maken van deze evolutie (met één control-qubit) verdubbelt weliswaar de benodigde twee-qubitpoorten voor elke term, maar bij 2 systeemqubits is dat beheersbaar. Kortom, H₂ vormt een ideaal testbed waar QPE in een simulatie-omgeving goed werkt en resultaten oplevert die dicht bij de exacte oplossing liggen.

Bij grotere moleculen neemt de complexiteit echter snel toe. **Ten eerste**, het aantal qubits groeit lineair met het aantal spinorbitalen (minus eventuele symmetrie- en core-reducties). Bijvoorbeeld, een molecuul als LiH of BeH₂ in minimale basis vergt al gauw 8–12 qubits na mapping, en complexe organische moleculen nog veel meer. **Ten tweede**, het aantal Pauli-termen in de Hamiltoniaan groeit combinatorisch met het aantal orbitale interacties (vier-indicem integralen leveren veel termen), wat de implementatie van $e^{-iHt}$ bemoeilijkt. De Trotterbenadering met één stap is mogelijk niet meer nauwkeurig genoeg; men zou meerdere Trotterstappen of geavanceerdere decomposities moeten gebruiken, wat het circuit proportioneel verdiept. Gecontroleerde evoluties over lange tijden (de $2^i$ factor kan groot worden bij benodigde hoge precisie) betekenen zeer lange reeksen quantumpoorten. **Ten derde**, de kwaliteit van de initiële toestand wordt kritischer. Voor grotere systemen is de HF-toestand vaak een minder goed uitgangspunt vanwege elektronencorrelatie. Het kan voorkomen dat de HF-state aanzienlijke overlap met hogere toestanden heeft, wat QPE-uitkomsten diffuus maakt. Men zou dan vooraf een betere benadering van de grondtoestand moeten voorbereiden (bijvoorbeeld via een variational eigensolver of aangepaste voorbereidingscircuits) zodat QPE van daaruit zuiverder kan werken. **Ten vierde**, decoherentie en foutcorrectie: QPE is een algoritme dat pas in het eindresultaat (na alle gecontroleerde evoluties en de QFT) de nuttige informatie oplevert. In een reëel kwantumapparaat zou het betekenen dat alle qubits gedurende de hele evolutie coherent moeten blijven. Voor grote moleculen en vereiste precisies kunnen QPE-circuits zo lang en complex zijn dat ze foutcorrectie vereisen. H₂ in simulatie heeft hier uiteraard geen last van – we simuleren foutloos – maar experimenteel QPE uitvoeren is momenteel alleen haalbaar voor zeer kleine circuits.

Om met deze uitdagingen om te gaan bij grotere moleculen, maakt men gebruik van technieken zoals **core-friezen** (alleen de chemisch actieve elektronen simuleren, zie FreezeCore), symmetriewegname (zoals de twee-qubit-reductie hier toegepast), en meer compacte Hamiltoniaanrepresentaties. Daarnaast wordt QPE in de literatuur vaak pas overwogen in combinatie met foutgecorrigeerde qubits, terwijl in het huidige NISQ-tijdperk variational quantum eigensolvers (VQE) populairder zijn voor moleculaire grondtoestanden juist omdat zij kortere circuits gebruiken. QPE blijft echter de gouden standaard als het gaat om uiteindelijk met polynomiale complexiteit hoge-precisie eigenenergieschattingen te krijgen, zodra de hardware het toelaat.

## Vergelijking met klassieke oplossingen

In het script wordt de QPE-uitkomst vergeleken met een klassieke referentie-oplossing voor verificatie. Die klassieke oplossing is verkregen via Qiskit’s `NumPyEigensolver` (of de verwante `NumPyMinimumEigensolver`), die in feite de Hamiltoniaanmatrix precies diagonaaliseert met lineaire algebra technieken. Voor H₂ is dit uitstekend te doen op een klassieke computer, aangezien de effectieve Hilbertruimte (2 qubits → $2^2=4$ dimensies na reductie) heel klein is. De klassieke solver geeft de exacte grondtoestandenergie (inclusief nucleaire repulsie indien men dat term optelt) terug, welke hier als “Exact Energy” wordt vermeld. We zien in de uitvoer bijvoorbeeld dat voor afstand 0,5 Å de klassieke grondtoestandenergie $-2.1135$ Hartree (exclusief kernrepulsie) is, terwijl QPE $-2.1396$ Hartree geeft. Dit verschil wordt ook in procent weergegeven als “Error”. Over het algemeen komt QPE redelijk overeen met de exacte oplossing, zeker rond de evenwichtsafstand. Kleinere afwijkingen analyseren we eerder reeds (HF-overlap en Trotterfout).

Het nut van deze vergelijking is tweeledig: (1) het valideert de QPE-implementatie op een bekend resultaat, en (2) het illustreert de potentie van QPE voor grotere problemen. Voor H₂ hebben we de luxe dat de volledige diagonaalprobleem klassiek oplosbaar is – iets wat bij grotere moleculen niet meer haalbaar is vanwege exponentiële schaal. De NumPy-oplosser dient hier dus als gouden standaard om te controleren of de quantumbenadering klopt. In de toekomst, voor moleculen met tientallen orbitaal, zal een klassieke diagonalisatie niet meer mogelijk zijn en zou een quantum phase estimation de eigenenergie kunnen vinden waar klassiek brute force faalt. In die zin demonstreert deze H₂-case de **correctheid** van QPE door terug te koppelen naar de klassieke oplossing, terwijl het bij grotere schaal de **noodzaak** van QPE benadrukt.

## Keuzes: ancilla-qubits en StatevectorSampler

Twee opmerkelijke instellingskeuzes in deze implementatie zijn het gebruik van **12 ancilla-qubits** en de `StatevectorSampler` als uitvoermethode. Ten eerste, het aantal ancilla’s bepaalt de precisie van de fase (en dus energie) die QPE kan onderscheiden. Met $m$ ancilla-qubits wordt de fase als een $m$-bits getal uitgelezen, wat overeenkomt met een resolutie van $2^{-m}$. In energie-termen is de resolutie $\Delta E \approx \frac{2\pi}{t} 2^{-m}$ (aangezien $E = -2\pi\phi/t$). Hier is $t=1$, dus $\Delta E \approx 2\pi/2^{12} \approx 0.0015$ (in natuurlijke units van het Hamiltoniaan, doorgaans Hartree voor moleculaire energieën). Met 12 qubits bereikt men dus een energieprecisie in de orde van $10^{-3}$ Hartree, wat rond de zogeheten chemische nauwkeurigheid ligt (\~$1.6\times10^{-3}$ Hartree). Dit rechtvaardigt de keuze van 12 ancilla’s: minder qubits zouden een grovere kwantisatie geven en potentieel significante afrondingsfouten in de gemeten energie. Meer qubits zouden uiteraard nog preciezer zijn, maar dat levert afnemende meerwaarde terwijl het circuit wel extra qubits en QFT-gates nodig heeft. De inverse QFT op 12 qubits is al van orde $\mathcal{O}(m^2) = 144$ two-qubit operaties (in een standaard implementatie), hoewel Qiskit’s library QFT mogelijk een geoptimaliseerde versie gebruikt. In elk geval is 12 qubits een ruime maar verantwoorde keuze om de grondtoestandenergie accuraat uit het fase-register te kunnen destilleren.

Ten tweede is de `StatevectorSampler` gebruikt om de circuituitvoering te simuleren. Deze Qiskit-primitieve berekent de toestandvector van het kwantumcircuit exact (zonder ruis of statistische fout) en genereert steekproeven alsof men met een ideaal quantumapparaat meet. Omdat we het circuit slechts op enkele qubits hebben (14 qubits totaal: 12 ancilla + 2 systeem) en een beperkt aantal poorten, is een volledige toestandvector-simulatie haalbaar. De code voert `sampler.run([qc], shots=10000)` uit, wat inhoudt dat er 10.000 metingen op de eindtoestand gesampled worden. In feite zou één enkele statevectorberekening volstaan om de theoretische waarschijnlijkheidsverdeling van de bitstrings te kennen, maar door shots te simuleren krijgen we een robuuste histogram-benadering van deze verdeling (vergelijkbaar met hoe een echt experiment data zou opleveren). Het gebruik van `StatevectorSampler` garandeert dat kwantumfouten, decoherentie of random fluctuaties geen rol spelen in deze analyse – men bekijkt de ideale QPE-prestatie. In plaats van de statevector-approach had men ook de `AerSimulator` (Qiskit’s dichtheidsmatrix- of Monte Carlo-simulator) kunnen gebruiken, maar het resultaat zou in dit ideale, ruisvrije geval hetzelfde zijn. De keuze voor `StatevectorSampler` is elegant, omdat het direct aansluit op Qiskit’s primitive interface en eenvoudig de gewenste `counts` oplevert, mits er geen tussentijdse metingen in het circuit zitten (deze sampler werkt alleen correct als alle metingen pas aan het einde plaatsvinden).

Samenvattend bieden 12 ancilla-qubits en het gebruik van een ideale statevectorsimulator de mogelijkheid om in deze H₂ QPE-demonstratie zeer precieze en betrouwbare resultaten te verkrijgen. Het laat zien dat, onder de juiste omstandigheden, QPE de grondtoestandenergie van een molecule kan benaderen met een nauwkeurigheid van beter dan een millihartree, in lijn met klassieke referenties. De aanpak werkt bijzonder goed voor H₂ dankzij de geringe schaal en eenvoud van het molecuul; tegelijkertijd illustreert het de elementen die van cruciaal belang zijn als men het algoritme naar grotere chemische systemen wil opschalen.

**Bronnen:** De implementatie is gebaseerd op Qiskit en Qiskit Nature modules, waaronder `ParityMapper` voor qubit-mapping, eventueel de `FreezeCoreTransformer` voor core-orbitalen, de `PauliEvolutionGate` voor tijdsevolutie, en Qiskit’s QPE-circuitopbouw met inverse QFT en meting. De resultaten zijn gevalideerd door vergelijking met Qiskit Nature’s klassieke eigensolver output. Deze aanpak en code zijn te vinden in de repositories van BramDo (`qiskit` en `qiskit-nature`) en in het bijgeleverde bestand `H2_qpe.pdf`, waarin de code en uitvoer voor verschillende H–H afstanden is weergegeven.
